In [1]:
import numpy as np

In [2]:
# # Compute the functional margin of an example (x,y) with respect to a hyperplane defined by w and b.
# def example_functional_margin(w, b, x, y):
#     result = y * (np.dot(w, x) + b)
#     return result

# # Compute the functional margin of a hyperplane for examples X with labels y.
# def functional_margin(w, b, X, y):
#     return np.min([example_functional_margin(w, b, x, y[i]) for i, x in enumerate(X)])

In [3]:
# x = np.array([1, 1])
# y = 1

# b_1 = 5
# w_1 = np.array([2, 1])

# w_2 = w_1 * 10
# b_2 = b_1 * 10

# print(example_functional_margin(w_1, b_1, x, y)) # 8
# print(example_functional_margin(w_2, b_2, x, y)) # 80

In [4]:
def example_geometric_margin(w, b, x, y):
    norm = np.linalg.norm(w)
    result = y * (np.dot(w/norm, x) + b/norm)
    return result

def geometric_margin(w, b, X, y):
    return np.min([example_geometric_margin(w, b, x, y[i]) for i, x in enumerate(X)])

In [5]:
x = np.array([1, 1])
y = 1

b_1 = 5
w_1 = np.array([2, 1])

w_2 = w_1 * 10
b_2 = b_1 * 10

print(example_geometric_margin(w_1, b_1, x, y)) # 8
print(example_geometric_margin(w_2, b_2, x, y)) # 80

3.577708763999664
3.577708763999664


In [6]:
# Compare two hyperplanes using the geometrical margin.
positive_x = [[2,7],[8,3],[7,5],[4,4],[4,6],[1,3],[2,5]]
negative_x = [[8,7],[4,10],[9,7],[7,10],[9,6],[4,8],[10,10]]

X = np.vstack((positive_x, negative_x))
y = np.hstack((np.ones(len(positive_x)), -1*np.ones(len(negative_x))))
w = np.array([-0.4, -1])
b = 8

print(X)
print(y)

# change the value of b
print(geometric_margin(w, b, X, y)) # 0.185695338177
print(geometric_margin(w, 8.5, X, y)) # 0.64993368362

[[ 2  7]
 [ 8  3]
 [ 7  5]
 [ 4  4]
 [ 4  6]
 [ 1  3]
 [ 2  5]
 [ 8  7]
 [ 4 10]
 [ 9  7]
 [ 7 10]
 [ 9  6]
 [ 4  8]
 [10 10]]
[ 1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1.]
0.18569533817705164
0.6499336836196807


In [7]:
from succinctly.datasets import get_dataset, linearly_separable as ls
import cvxopt.solvers

In [8]:
X, y = get_dataset(ls.get_training_examples)
m = X.shape[0]

In [9]:
K = np.array([np.dot(X[i], X[j]) for j in range(m) for i in range(m)]).reshape((m, m))

In [10]:
P = cvxopt.matrix(np.outer(y, y) * K)
q = cvxopt.matrix(-1 * np.ones(m))

# Equality constraints
A = cvxopt.matrix(y, (1, m))
b = cvxopt.matrix(0.0)

# Inequality constraints
G = cvxopt.matrix(np.diag(-1 * np.ones(m)))
h = cvxopt.matrix(np.zeros(m))

# Solve the problem
solution = cvxopt.solvers.qp(P, q, G, h, A, b)

# Lagrange multipliers
multipliers = np.ravel(solution['x'])

# Support vectors have positive multipliers.
has_positive_multiplier = multipliers > 1e-7
sv_multipliers = multipliers[has_positive_multiplier]
support_vectors = X[has_positive_multiplier]
support_vectors_y = y[has_positive_multiplier]

     pcost       dcost       gap    pres   dres
 0: -3.9356e+00 -7.2072e+00  4e+01  6e+00  2e+00
 1: -5.9831e+00 -4.3032e+00  1e+01  2e+00  6e-01
 2: -5.6350e-01 -1.1535e+00  2e+00  1e-01  4e-02
 3: -6.2758e-01 -7.4538e-01  1e-01  2e-16  9e-15
 4: -7.1507e-01 -7.1641e-01  1e-03  1e-16  1e-14
 5: -7.1604e-01 -7.1605e-01  1e-05  2e-16  6e-15
 6: -7.1605e-01 -7.1605e-01  1e-07  2e-16  9e-15
Optimal solution found.


In [11]:
def compute_w(multipliers, X, y):
    return np.sum(multipliers[i] * y[i] * X[i] for i in range(len(y)))

In [12]:
w = compute_w(multipliers, X, y)
w_from_sv = compute_w(sv_multipliers, support_vectors, support_vectors_y)

print(w)
print(w_from_sv) 

[0.44444446 1.11111114]
[0.44444453 1.11111128]


C:\Users\hyeon\AppData\Local\Temp\ipykernel_14728\905609691.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(multipliers[i] * y[i] * X[i] for i in range(len(y)))


In [13]:
def compute_b(w, X, y):
    return np.sum([y[i] - np.dot(w, X[i]) for i in range(len(X))])/len(X)

In [14]:
b = compute_b(w, support_vectors, support_vectors_y)

In [15]:
b

-9.666666925153795

In [16]:
w = np.array([0.4, 1])
b = -10
x = np.array([6, 8])
y = -1

In [17]:
def constraint(w, b, x, y):
    return y * (np.dot(w, x) + b)

In [18]:
def hard_constraint_is_satisfied(w, b, x, y):
    return constraint(w, b, x, y) >= 1

def soft_constraint_is_satisfied(w, b, x, y, zeta):
    return constraint(w, b, x, y) >= 1 - zeta

In [19]:
# While the constraint is not satisfied for the example (6,8).
print(hard_constraint_is_satisfied(w, b, x, y)) # False

# We can use zeta = 2 and satisfy the soft constraint.
print(soft_constraint_is_satisfied(w, b, x, y, zeta=2)) # True

False
True


In [20]:
constraint(w, b, x, y)

-0.40000000000000036

In [21]:
# We can pick a huge zeta for every point to always satisfy the soft constraint.
print(soft_constraint_is_satisfied(w, b, x, y, zeta=10)) # True
print(soft_constraint_is_satisfied(w, b, x, y, zeta=1000)) # True

True
True


In [22]:
# Transform a two-dimensional vector x into a three-dimensional vector.
def transform(x):
    return [x[0]**2, np.sqrt(2)*x[0]*x[1], x[1]**2]

In [23]:
def polynomial_kernel(a, b, degree, constant=0):
    result = sum([a[i] * b[i] for i in range(len(a))]) + constant
    return pow(result, degree)

In [24]:
x1 = [3,6]
x2 = [10,10]
# We do not transform the data.
print(polynomial_kernel(x1, x2, degree=2)) # 8100


8100


In [25]:
def kernel(x1, x2):
    return np.dot(x1, x2.T)

def objective_function_to_minimize(X, y, a, kernel):
    m, n = np.shape(X)
    return 1 / 2 * np.sum([a[i] * a[j] * y[i] * y[j]* kernel(X[i, :], X[j, :]) for j in range(m) for i in range(m)]) - np.sum([a[i] for i in range(m)])

In [26]:
# def get_non_bound_indexes(self):
#     return np.where(np.logical_and(self.alphas > 0, self.alphas < self.C))[0]

In [27]:
# # First heuristic: loop over examples where alpha is not 0 and not C they are the most likely to violate the KKT conditions (the non-bound subset).
# def first_heuristic(self):
#     num_changed = 0
#     non_bound_idx = self.get_non_bound_indexes()
#     for i in non_bound_idx:
#         num_changed += self.examine_example(i)
#     return num_changed